# MLP Coding Example

Acesse o servidor remoto por ssh. Crie uma virtualenv com:
```
mkvirtualenv <nome-da-sua-env>
```
Ative a sua virtualenv com:
```
workon <nome-da-sua-env>
```
Instale o jupyter:
```
pip install jupyter
```
Na pasta contendo o setup.py, instale o pacote do projeto :
```
pip install -e .
```
Comando para servir o jupyter:
```
nohup jupyter notebook --no-browser &
```
Talvez você precise de um token. Se precisar consulte com:
```
jupyter notebook list
```




Na sua máquina local, redirecione a porta adequada:
```
ssh -NfL localhost:<porta-local>:localhost:<porta-remoto> <seu-usuario>@<ip-do-servidor>
```
Geralmente:
```
ssh -NfL localhost:8888:localhost:8888 <seu-usuario>@<ip-do-servidor>
```
Abra localhost:8888 no seu browser. Se você quiser fechar o jupyter, no localhost:8888 clique em Quit, depois libere a porta com:
```
lsof -ti:8888 | xargs kill -9
```

## Imports

In [1]:
import numpy as np
from numpy import genfromtxt
from tqdm.notebook import tqdm
from perceptronac.context_training import context_training
from perceptronac.context_coding import context_coding
from perceptronac.perfect_AC import perfect_AC
import torch
import os

## Gerando dados randômicos correlacionados (substituir pelos seus dados)

In [2]:
#X is context and y is bitstream to encode
# parameters  
L = 100000 # how many samples 
N = 7 # order of the AR
# Np = N # number of parameters to estimate 

C0 = np.random.rand(1,1) 
C = np.random.rand(N,1)

X = 2 * (np.random.rand(2*L,N) > 0.5) - 1 # correlated (context) signals

X = (X > 0).astype(int)

def sigmoid(x): 
    return 1 / (1 + np.e**(-x))

p = sigmoid(C0 + X @ C);
yy = (np.random.rand(2*L, 1) > (1 - p)).astype(int) # signal 
yt = yy[0:L] > 0 # train on the first part 
yc = yy[L:L+L] > 0 # encode the second part
Xt = X[0:L,0:N] # truncated X for training 
Xc = X[L:L+L,0:N] # truncated X for coding

In [3]:
#Xc

In [8]:
#X is context and y is bitstream to encode
L = 100000 # how many samples 
N = 7 # order of the AR
my_data = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo28_v3.csv', delimiter=',')
bitstream = my_data[:, 0].astype(int)
context = my_data[:, 1:6].astype(float)
extra_context = my_data[:, 7:].astype(float)
#extra_context = extra_context.reshape((len(extra_context), 1))
bitstream = bitstream.reshape((len(bitstream), 1))


my_data_test = genfromtxt('../SPIHT_dataset/SPIHT_bits_with_context_ricardo29_v3.csv', delimiter=',')
bitstream_test = my_data_test[:, 0].astype(int)
context_test = my_data_test[:, 1:6].astype(float)
extra_context_test = my_data_test[:, 7:].astype(float)
#extra_context_test = extra_context_test.reshape((len(extra_context_test), 1))
bitstream_test = bitstream_test.reshape((len(bitstream_test), 1))


yt = bitstream > 0 # train on the first part 
yc = bitstream_test > 0 # encode the second part
Xt = context >= 0 # truncated X for training 
Xc = context_test >= 0 # truncated X for coding
Xt = Xt.astype(int)
Xc = Xc.astype(int)
Xt = np.append(Xt, extra_context, axis=1)
Xc = np.append(Xc, extra_context_test, axis=1)

In [9]:
Xc[0,:]

array([1. , 0. , 0. , 0. , 0. , 0. , 0.5])

## Entropia dos dados

In [10]:
# treino
perfect_AC(yt,context_coding(Xt,context_training(Xt,yt)))

0.8961499765246234

In [11]:
# teste
perfect_AC(yc,context_coding(Xc,context_training(Xc,yc)))

0.8974801400717474

# Criando classes

In [12]:
import torch

In [13]:
class Perceptron(torch.nn.Module):
    def __init__(self,N):
        super().__init__()
        self.linear = torch.nn.Linear(N, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [14]:
class Log2BCELoss(torch.nn.Module):
    def __init__(self,*args,**kwargs):
        super().__init__()
        self.bce_loss = torch.nn.BCELoss(*args,**kwargs)

    def forward(self, pred, target):
        return self.bce_loss(pred, target)/torch.log(torch.tensor(2,dtype=target.dtype,device=target.device))

In [15]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        return self.X[idx,:],self.y[idx,:]

## Treinando Modelo No Pytorch com Batch Gradient Descent (Quando todos os dados couberem na memória da placa de vídeo de uma só vez)

In [61]:
net = Perceptron(N)

In [62]:
net = net.cuda()

In [63]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [64]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)

In [65]:
train_loss = []
for epoch in range(20000):
    optimizer.zero_grad()
    outputs = net(torch.tensor(trainset.X).float().cuda())
    loss = criterion(outputs,torch.tensor(trainset.y).view(-1,1).float().cuda())
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item()/len(trainset))

In [66]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

comprimento médio de código final no dataset de treino: 0.003491003723144531
(compare com a entropia do dataset de treino).


### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [67]:
for param in net.parameters():
    print(param.data)

tensor([[12.0901, 11.7186, 11.9979, 12.1763, 12.1026]], device='cuda:0')
tensor([41.8704], device='cuda:0')


In [68]:
C.T, C0

(array([[0.32098451, 0.77295341, 0.82311742, 0.86189127, 0.7005142 ,
         0.80153656, 0.21184858]]),
 array([[0.2792105]]))

## Treinando Modelo No Pytorch com Stochastic Gradient Descent (um pedaço dos dados na memória da placa de vídeo de cada vez)

In [16]:
net = Perceptron(N)

In [17]:
net = net.cuda()

In [18]:
trainset = CustomDataset(Xt,yt)
validset = CustomDataset(Xc,yc)

In [19]:
criterion = Log2BCELoss(reduction="sum")
optimizer = torch.optim.SGD(net.parameters(), lr=0.00001)
#optimizer = torch.optim.SGD(net.parameters(), lr=0.000001)

In [ ]:

batch_size = 100000

train_loss, valid_loss = [], []

for epoch in range(300):  # loop over the dataset multiple times

    for phase in ['train','valid']:

        if phase == 'train':
            net.train(True)
            dataloader = torch.utils.data.DataLoader(
                trainset,batch_size=batch_size,shuffle=True,num_workers=2)
        else:
            net.train(False)
            dataloader=torch.utils.data.DataLoader(
                validset,batch_size=batch_size,shuffle=False,num_workers=2)
            
        running_loss = 0.0
        for data in dataloader: #tqdm(dataloader):
            
            X_b,y_b= data
            X_b = X_b.float().cuda()
            y_b = y_b.float().cuda()
            
            if phase == 'train':
                optimizer.zero_grad()
                outputs = net(X_b.float())
                loss = criterion(outputs,y_b.view(-1,1).float())
                loss.backward()
                optimizer.step()
            else:
                with torch.no_grad():
                    outputs = net(X_b)
                    loss = criterion(outputs, y_b)

            running_loss += loss.item()

        final_loss = running_loss / len(dataloader.dataset)
        if phase=='train':
            train_loss.append(final_loss)
        else:
            valid_loss.append(final_loss)
            
        print("epoch :" , epoch, ", phase :", phase, ", loss :", final_loss)

print('Finished Training')

epoch : 0 , phase : train , loss : 36.414972073497864
epoch : 0 , phase : valid , loss : 38.97983027316965
epoch : 1 , phase : train , loss : 33.92692110398681
epoch : 1 , phase : valid , loss : 18.174040159914743
epoch : 2 , phase : train , loss : 14.608137556246287
epoch : 2 , phase : valid , loss : 9.344935176644658
epoch : 3 , phase : train , loss : 29.98220732266315
epoch : 3 , phase : valid , loss : 16.141568551408316
epoch : 4 , phase : train , loss : 28.952730402966168
epoch : 4 , phase : valid , loss : 26.381696869328124
epoch : 5 , phase : train , loss : 17.66749824039643
epoch : 5 , phase : valid , loss : 7.538301044504013
epoch : 6 , phase : train , loss : 19.68650433221858
epoch : 6 , phase : valid , loss : 50.28031807128541
epoch : 7 , phase : train , loss : 50.14528782232276
epoch : 7 , phase : valid , loss : 50.22697743016578
epoch : 8 , phase : train , loss : 50.10003702287281
epoch : 8 , phase : valid , loss : 50.18922151112938
epoch : 9 , phase : train , loss : 50.06

epoch : 76 , phase : train , loss : 44.00915688883103
epoch : 76 , phase : valid , loss : 41.67472264448995
epoch : 77 , phase : train , loss : 41.27775461022916
epoch : 77 , phase : valid , loss : 40.81285248460225
epoch : 78 , phase : train , loss : 39.71493133816665
epoch : 78 , phase : valid , loss : 37.76153782692313
epoch : 79 , phase : train , loss : 36.15138286532625
epoch : 79 , phase : valid , loss : 33.086108691712404
epoch : 80 , phase : train , loss : 28.202116575940288
epoch : 80 , phase : valid , loss : 20.64365877664682
epoch : 81 , phase : train , loss : 12.944895017948635
epoch : 81 , phase : valid , loss : 6.941043949504595
epoch : 82 , phase : train , loss : 36.48994211955269
epoch : 82 , phase : valid , loss : 40.43227122481383
epoch : 83 , phase : train , loss : 38.41628084222289
epoch : 83 , phase : valid , loss : 33.37855730503629
epoch : 84 , phase : train , loss : 25.98971954156733
epoch : 84 , phase : valid , loss : 4.709938804193028
epoch : 85 , phase : trai

In [ ]:
print(f"""comprimento médio de código final no dataset de treino: {train_loss[-1]}
(compare com a entropia do dataset de treino).""")

print(f"""comprimento médio de código final no dataset de validação: {valid_loss[-1]}
(compare com a entropia do dataset de validação).""")

### Pesos aprendidos são aproximadamente os parâmetros usados para gerar os dados

In [ ]:
for param in net.parameters():
    print(param.data)

In [ ]:
C.T, C0